In [ ]:
%matplotlib notebook

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
import numpy as np
from disba import PhaseDispersion, GroupDispersion
from disba import depthplot
from Model import Model

In [ ]:
periods = np.logspace(0.0, 3., 100)
periods = periods[periods < 700]

#f = np.linspace(0.04, 0.18, 30)
#periods = 1/f[::-1]

In [ ]:
prem = Model("/usr/local/TauP-2.1.1/StdModels/prem.nd")

In [ ]:
f , (ax1, ax2) = plt.subplots(1,2, figsize = (8,4))

f = depthplot(*prem.thick_get('vs'),
              plot_args = { 'label' : 'Vs', 'ls' :'-', 'color' : 'red' },
              ax = ax1)

f = depthplot(*prem.thick_get('vp'),
              plot_args = { 'label' : 'Vp', 'ls' :'-', 'color' : 'red' },
              ax = ax2)


for d in np.arange(70,200,20):
    _m = prem.mod({
        'l' : (None, d, True),
        'u' : (d, None, True)    
    })
    f = depthplot(*_m.thick_get('vs'),
                  plot_args = { 'label' : 'Vs', 'ls':'--', 'color' : 'gray', 'linewidth' : 1},
                  ax = ax1)
    f = depthplot(*_m.thick_get('vp'),
                  plot_args = { 'label' : 'Vp','ls':'--', 'color' : 'gray', 'linewidth' : 1 },
                  ax = ax2)

ax1.set_title('S-Wave')
ax1.set_ylim([410,0])
ax1.set_xlim([4.2,4.8])

ax2.set_ylim([410,0])
ax2.set_xlim([7.5,9])
ax2.set_title('P-Wave')

In [ ]:
_,_,f_per,_,f_C,f_U,_,_,_ = np.loadtxt("prem-R.asc", skiprows = 1, unpack = True)
_,_,fo_per,_,fo_C,fo_U,_,_,_ = np.loadtxt("prem-ocean-R.asc", skiprows = 1, unpack = True)

#_,_,f_per,_,f_C,f_U,_,_ = np.loadtxt("prem-L.asc", skiprows = 1, unpack = True)
#_,_,fo_per,_,fo_C,fo_U,_,_ = np.loadtxt("prem-ocean-L.asc", skiprows = 1, unpack = True)

In [ ]:
# MM = 2850
MM = 2850

d,vp,vs,rho = prem.thick_get(['vp', 'vs', 'rho'], max_depth = MM)
GpremF  = GroupDispersion(d, vp, vs, rho, model_type = 'spherical')
Rg_premF = GpremF(periods, 0, 'rayleigh')

In [ ]:
f , (ax1, ax2) = plt.subplots(1,2, figsize = (8,4))

f = depthplot(*prem.thick_get('vp'),
              plot_args = { 'label' : 'Vs' } ,
              ax = ax1)

ax1.axhspan(0.0, MM if MM is not None else 6371, alpha = 0.2)

ax2.plot(Rg_premF.period, Rg_premF.velocity, 'o', label = 'PREM - Flat')

ax2.plot(f_per, f_C, label = 'Herman C')
ax2.plot(f_per, f_U, label = 'Herman U')

ax2.plot(fo_per, fo_C, label = 'Herman Ocean C')
ax2.plot(fo_per, fo_U, label = 'Herman Ocean U')


ax1.set_xlabel('S-wave velocity [km/s]')
ax1.set_ylabel('Depth [km]')

ax2.set_xscale("log", base=10)
#ax2.set_yscale("log", base=10)

ax2.set_xlabel('Period [s]')
#ax2.set_xlim(1,100)
ax2.set_ylabel('Velocity [km/s]')

ax2.legend()

plt.tight_layout()